In [1]:
#使用Ollama模型
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from typing import List
from typing import TypedDict
from langchain.schema import Document
from langgraph.graph import END, StateGraph, START
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

os.environ["LANGCHAIN_TRACING_V2"] = "true"
#https://smith.langchain.com/o/d1c9fe9f-3a8e-5df2-812b-f1dbc714d565/projects/p/479b7e6d-598a-4a5f-9a5c-515d7c50545d?timeModel=%7B%22duration%22%3A%227d%22%7D
os.environ["LANGCHAIN_API_KEY"] = "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" #trace
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
#https://app.tavily.com/?code=gWC496d2-HPgVCefwjLyPq9O2JKr8D1E9Wr9T--frPf-h&state=eyJyZXR1cm5UbyI6Imh0dHBzOi8vYXBwLnRhdmlseS5jb20ifQ
os.environ["TAVILY_API_KEY"] = "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" #web search


In [2]:
# 初始化語言模型和嵌入模型並建立RAG
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
llm = ChatOllama(model="llama3.1", temperature=0.6)

# 加載Chroma
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda:0'}
encode_kwargs = {'normalize_embeddings': True}
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

#關於英濟
Profile_directory = 'DB_En_add_tw/MGF_Profile'
Profile_vectordb = Chroma(persist_directory=Profile_directory, embedding_function=embedding_model)
Profile_retriever = Profile_vectordb.as_retriever(search_kwargs={"k": 10})
#英濟事業
Product_directory = 'DB_En_add_tw/MGF_Product'
Product_vectordb = Chroma(persist_directory=Product_directory, embedding_function=embedding_model)
Product_retriever = Profile_vectordb.as_retriever(search_kwargs={"k": 10})

#技術發展
Technology_directory = 'DB_En_add_tw/MGF_Technology'
Technology_vectordb = Chroma(persist_directory=Technology_directory, embedding_function=embedding_model)
Technology_retriever = Technology_vectordb.as_retriever(search_kwargs={"k": 10})


/home/swrd/iwb_proj/iwb/NLP_API/ting/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/swrd/iwb_proj/iwb/NLP_API/ting/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
###建立要從vectorstore還是web_search的判斷器
class RouteQuery(BaseModel):
    datasource: Literal["vectorstore", "web_search"] = Field(
        ...,
        description="Given a user question choose to route it to web search or a vectorstore.",
    )
structured_llm_router = llm.with_structured_output(RouteQuery)    
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """
         You are an expert responsible for routing user questions to either a vectorstore or web search. 
         The vectorstore contains documents related to Megaforce, including information on its business and technological developments. 
         For questions on these topics, use the vectorstore. Otherwise, use web search.
         """
        ),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router

### 檢索評分器
class GradeDocuments(BaseModel):
    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

structured_llm_grader = llm.with_structured_output(GradeDocuments)
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """
         You are a grader assessing relevance of a retrieved document to a user question. \n 
         If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
         It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
         Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question.
         """
        ),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)
retrieval_grader = grade_prompt | structured_llm_grader

###Generate
Generate_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """
         You are an assistant for question-answering tasks.
         Use the following pieces of retrieved context to formulate your answer.
         If the context does not provide sufficient information to answer the question, clearly state that you don't know.
         Ensure your response includes at least one explanatory sentence or piece of information.
         Generate the answer in a professional customer service style.
         Question: {question}
         Context: {context}
         Please respond in either English or Traditional Chinese, choosing just one language:
         """
        ),
        ("human", "{question}"),
    ]
)



# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = Generate_prompt | llm | StrOutputParser()

###幻覺評分器
class GradeHallucinations(BaseModel):
    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )

# LLM with function call
structured_llm_Hall = llm.with_structured_output(GradeHallucinations)
# Prompt
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """
         You are a grader assessing whether an LLM-generated response is reasonably consistent with or loosely supported by a set of retrieved facts.
         Provide a binary score: 'yes' or 'no'.
         A 'yes' indicates that the response is generally plausible based on the facts, even if there are significant discrepancies, omissions, or speculative assumptions, as long as the overall message is coherent and not blatantly contradicted by the facts.
         """
        ),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)


hallucination_grader = hallucination_prompt | structured_llm_Hall

###回答評分器
class GradeAnswer(BaseModel):
    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )
structured_llm_Ans = llm.with_structured_output(GradeAnswer)
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """
         You are a grader assessing whether an answer sufficiently addresses or relates to a question. 
         The bar is relaxed, so as long as the answer is somewhat relevant or provides partial resolution, 
         give a binary score 'yes'. A 'yes' means that the answer resolves or partially addresses the question. 
         Only give 'no' if the answer is completely off-topic or irrelevant.
         """
        ),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_Ans

### Question Re-writer
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """
         You a question re-writer that converts an input question to a better version that is optimized \n 
         for vectorstore retrieval. 
         Please rewrite the input question into an optimized version that is more suitable for vectorstore retrieval, while retaining the original question.
         Observe the input content and try to infer its underlying semantic intent/meaning.
         """
        ),
        ("human","Here is the initial question: \n\n {question} \n Formulate an improved question.",),
    ]
)
question_rewriter = re_write_prompt | llm | StrOutputParser()


In [7]:
#Graph state
class GraphState(TypedDict):
    question: str
    generation: str
    documents: List[str]
    transform_query_count:int
    previous_node: str
    
#Retrieve documents
def retrieve(state):
    print("---RETRIEVE---")
    question = state["question"]
    previous_node = state.get("previous_node")
    if previous_node:
        if previous_node == "Profile_retrieve":
            retriever = Profile_retriever
        elif previous_node == "Product_retrieve":
            retriever = Product_retriever
        elif previous_node == "Technology_retrieve":
            retriever = Technology_retriever
    else:  # 如果 previous_node 為 None
        retriever, previous_node = select_retriever(question)
    
    if retriever is None:
        # 如果沒有找到合適的 retriever，直接返回並跳轉到 'web_search'
        return {"next_node": "web_search", "question": question, "previous_node": "web_search"}
    
    # Retrieval
    documents = retriever.invoke(question)
    
    return {"documents": documents, "question": question, "previous_node": previous_node}

def select_retriever(question):
    # 根據問題內容選擇檢索器
    if "profile" in question.lower():
        print("---PROFILE RETRIEVER SELECTED---")
        return Profile_retriever, "Profile_retrieve"
    
    elif "product" in question.lower():
        print("---PRODUCT RETRIEVER SELECTED---")
        return Product_retriever, "Product_retrieve"
    
    elif "technology" in question.lower():
        #print("---TECHNOLOGY RETRIEVER SELECTED---")
        return Technology_retriever, "Technology_retrieve"
    else:
        # 當沒有適合的檢索器時，返回 'web_search'
        print("---NO SUITABLE RETRIEVER FOUND, RETURNING TO WEB SEARCH---")
        return None, "web_search"

#Generate answer
def generate(state):
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation with max_tokens for OllamaFunction
    generation = rag_chain.invoke({"context": documents,"question": question })
 
    return {"documents": documents, "question": question, "generation": generation}

#評估檢索到的文檔是否與給定問題相關
def grade_documents(state):
    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}

#Transform the query to produce a better question
def transform_query(state):
    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]
    query_count = (state.get('transform_query_count', 0) or 0) + 1
    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question, "transform_query_count":query_count}

#web Search
web_search_tool = TavilySearchResults(k=3)

def web_search(state):
    print("---WEB SEARCH---")
    question = state["question"]
    previous_node = state.get("previous_node", "web_search")
    print(question)
    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)

    return {"documents": web_results, "question": question, "previous_node": previous_node}
#Edges
#選擇web search or RAG
def route_question(state):
    print("---ROUTE QUESTION---")
    question = state["question"]
    source = question_router.invoke({"question": question})
    if source.datasource == "web_search":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "web_search"
    elif source.datasource == "vectorstore":
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"
    
# transform_query完後決定跳哪個節點   
def decide_node(state):
    print("---DECIDING NEXT NODE---")
    # 檢查上一次的節點
    previous_node = state.get("previous_node")
    if previous_node == "web_search":
        print("Previous node was web_search, returning to web_search.")
        return "web_search"
    else:
        print("Previous node was not web_search, proceeding to retrieve.")
        return "retrieve"
    
#生成或重新詢問    
def decide_to_generate(state):
    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    transform_query_count = state.get("transform_query_count", 0)
    filtered_documents = state["documents"]
    if transform_query_count <=5:
        if not filtered_documents:
            # All documents have been filtered check_relevance
            # We will re-generate a new query
            print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---")
            return "transform_query"
        else:
            # We have relevant documents, so generate answer
            print("---DECISION: GENERATE---")
            return "generate"
    else:
        print("---QUERY COUNT OVER 5, SO DECISION: GENERATE---")
        return "generate"
    
#確定生成的內容是否基於文檔並回答了問題
def grade_generation_v_documents_and_question(state):
    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]
    # 檢查documents是否可以迭代
    if hasattr(documents, '__len__'):
        for i, doc in enumerate(documents):
            print(f"Processing document {i + 1} of {len(documents)}")
            # 單個文件和生成的回答配對進行評估
            score = hallucination_grader.invoke({"documents": [doc], "generation": generation})
            grade = score.binary_score
            
            # Check hallucination
            if grade == "yes":
                print("---決策: 生成內容是基於文件的---")
                # Check question-answering
                print("---評分生成內容 vs 問題---")
                score = answer_grader.invoke({"question": question, "generation": generation})
                grade = score.binary_score
                
                if grade == "yes":
                    print("---DECISION: GENERATION ADDRESSES QUESTION---")
                    return "useful"
                else:
                    print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
                    return "not useful"

        # 如果遍歷完所有文檔後仍未返回，則表示所有文檔都未通過幻覺評分
        print("---DECISION: GENERATION IS NOT GROUNDED IN THIS DOCUMENT, CONTINUING---")
        return "not supported"
    else:
        score = hallucination_grader.invoke({"documents": [documents], "generation": generation})
        grade = score.binary_score
        # Check hallucination
        if grade == "yes":
            print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
            # Check question-answering
            print("---GRADE GENERATION vs QUESTION---")
            score = answer_grader.invoke({"question": question, "generation": generation})
            grade = score.binary_score
            if grade == "yes":
                print("---DECISION: GENERATION ADDRESSES QUESTION---")
                return "useful"
            else:
                print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
                return "not useful"
        else:
            print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
            return "not supported"

In [8]:
workflow = StateGraph(GraphState)

workflow.add_node("web_search", web_search)  
workflow.add_node("retrieve", retrieve)  
workflow.add_node("grade_documents", grade_documents)  
workflow.add_node("generate", generate) 
workflow.add_node("transform_query", transform_query) 
workflow.add_conditional_edges(
    START,
    route_question,
    {
        "web_search": "web_search",
        "vectorstore": "retrieve",
    },)
workflow.add_edge("web_search", "generate")
workflow.add_conditional_edges(
    "retrieve",
    lambda state: state.get("next_node", "grade_documents"),  # 檢查下一個節點
    {
        "web_search": "web_search",  # 如果返回的 next_node 是 'web_search'，則跳轉到 web_search
        "grade_documents": "grade_documents",  # 否則繼續流程
    })

workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },)
workflow.add_conditional_edges(
    "transform_query",
    decide_node,
    {
        "retrieve" : "retrieve",
        "web_search": "web_search"
    }, )
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
    },)

app = workflow.compile()


In [ ]:
# Run
def get_new_model_instance():
    # 每次提問前重新初始化模型
    return workflow.compile()
history = []
# Run
input_text = input('>>> ')
history = []
while input_text.lower() != 'bye':
    app = get_new_model_instance()  # 重置模型上下文
    inputs = {"question": input_text}
    # 模型輸出
    for output in app.stream(inputs):
        for key, value in output.items():            
            generation = value.get("generation", "No response generated.")
    history.append({"question": input_text, "answer": generation})
    input_text = input('>>> ')


In [ ]:
# 從輸入資料中獲取文件和生成內容
documents = [
    Document(page_content='技術研發\nTechnology R & D\n創新研發中心\n創新研發中心支援產業生態圈\n2024/9/12 上午9:13\n英濟 - 技術研發\nhttps://www.megaforce.com.tw//zh-tw/Tech/Technology\n1/6\n'),
    Document(page_content='more\n完整模修能力\n先進高精密模具製造\n高品質精密量測設備\n高精密模具加工設備\n優異品質掌控設備\n高端成型技術\n高端成型技術\n除各類精密塑膠零組件設計與射出成型外，英濟亦持續發展高值LSR液態矽\n膠成型，不僅是單獨矽膠成型與矽塑膠結合成型量產，更進一步發展矽膠金\n屬結合成型技術及矽膠雙射成型技術，而矽膠包覆中空成型技術更是領先業\n界的創新開發。\n順應愈趨多元的產品應用場景與需求，英濟將技術領域擴展至各類多元材質\n與異材質結合等層面，協助客戶開發更多元新商品的可能性。\n2024/9/12 上午9:13\n英濟 - 技術研發\nhttps://www.megaforce.com.tw//zh-tw/Tech/Technology\n3/6\n'),
    Document(page_content='技術研發\nTechnology R & D\nInnovative R&D Centers\nInnovative R&D Centers\nSupport Industrial Ecosystem\n2024/9/12 上午9:16\nMEGAFORCE - Technology\nhttps://www.megaforce.com.tw//en-global/Tech/Technology\n1/6\n'),
    Document(page_content='more\n選擇性膠黏劑、光固化接著劑\n表面處理劑(Primer)\n導熱材料(墊片與膠)\n異材質結合技術\n高分子實驗室\n高分子實驗室\n高分子實驗室成立於2002年，配備各類先進精密儀器，如：GC-MS、\nICP、DSC、SEM等。實驗室管理完善、檢測能力不斷提升，已經形成了完\n整有效地管理體系，並于2007年通過了符合CNAS（ISO 17025）檢測和校\n準實驗室能力認可準則的管理體系。\n2024/9/12 上午9:13\n英濟 - 技術研發\nhttps://www.megaforce.com.tw//zh-tw/Tech/Technology\n5/6\n'),
    Document(page_content='more\n雙/多射出成型技術\n塑件包覆LSR成型\n液態矽膠(LSR)成型/多層複合成型技術\n精密埋射技術\n金屬包覆LSR成型\n材料應用技術\n材料應用技術\n為了更完善應對產品生產過程中的影響因子，英濟亦對材料與各製程中所需\n技術積極投入研發，包含表面質感、材質結合、導熱技術等，結合數十項專\n利構築的技術優勢，創造具差異化的競爭力，提供客戶創新材料的資源應\n用，促使其發展更全面應用。\n矽膠手感噴塗劑\n2024/9/12 上午9:13\n英濟 - 技術研發\nhttps://www.megaforce.com.tw//zh-tw/Tech/Technology\n4/6\n'),
    Document(page_content='more\nService@megaforce.com.tw\n新北市土城區自強街5號2樓\n+886-2-2268-7790\n2016 © MEGAFORCE. All Rights Reserved 法律聲明與使用條款 網站地圖\n2024/9/12 上午9:13\n英濟 - 技術研發\nhttps://www.megaforce.com.tw//zh-tw/Tech/Technology\n6/6\n'),
    Document(page_content='技術研發\nTechnology R & D\n高端成型技術\n液態射出矽膠\n英濟自行研發的選擇性黏著LSR(Liquid Silicone Rubber)，對PC塑料具\n有良好的黏著性，且不黏金屬、不易沾模，適用於注射成型、熱壓成\n型、異材質結合。產品精確度高，具優異的抗撕裂強度、回彈性、抗黃\n變性、耐熱老化性、熱穩定性和耐候性，極具替代工程軟膠的潛力。材\n料應用層面廣，涵蓋消費電子產品、戶外運動產品、攜帶型智慧裝置\n等。集團擁有實驗室及研發團隊，可視客戶需求調整材料性能。\n多射射出成型\n在既有的射出成型機上，外掛一組~多組副射台，並搭配精密設計的模仁\n旋轉機構，便可在一個行程中產出多色或異材結合的產品，滿足多變化\n的工藝需求。 不僅大大的提升了不同材質間的結合強度，在於工程數的\n減少，成型周期減少了，以往多段工程的數套模具，現在也只需要一套\n設計完美的模具即可完成，因此模具數也減少了，成本也跟著降低。\nLSR M+R射出成型\n金屬材質應用於消費性電子產品日益增加，但因碰撞與摔落易造成外觀\n損傷。藉由LSR成型技術，M+R(金屬+矽膠)設計可增加產品外觀與機構\n防護。 雙材質設計，兼顧外觀質感與觸感，進一步提升產品價值。\n2024/9/12 上午9:14\n英濟 - 技術研發\nhttps://www.megaforce.com.tw//zh-tw/Tech/Technology/List/1\n1/2\n')
]


generation = '根據提供的上下文，我們可以看到英濟公司的技術研發範圍非常廣泛，包括：\n\n* 高精密模具製造\n* 精密量測設備\n* 高精密模具加工設備\n* 優異品質掌控設備\n* 高端成型技術\n* 液態矽膠(LSR)成型/多層複合成型技術\n* 精密埋射技術\n* 金屬包覆LSR成型\n* 材料應用技術\n* 矽膠手感噴塗劑\n\n另外，英濟公司還有高分子實驗室，配備先進精密儀器，如GC-MS、ICP、DSC、SEM等。實驗室管理完善，檢測能力不斷提升。\n\n在技術研發方面，英濟公司也發展了多射出成型技術，能夠在一個行程中產出多色或異材結合的產品。\n\n總之，英濟公司的技術研發有許多創新和先進的技術，涵蓋模具製造、精密量測、材料應用等各個方面。'

# 依序處理每個文件
for i, doc in enumerate(documents):
    print(f"Processing document {i+1} of {len(documents)}")
    
    # 單個文件和生成的回答配對進行評估
    score = hallucination_grader.invoke({"documents": [doc], "generation": generation})
    print(score)


Processing document 1 of 7
binary_score='no'
Processing document 2 of 7
binary_score='yes'
Processing document 3 of 7
binary_score='yes'
Processing document 4 of 7
binary_score='yes'
Processing document 5 of 7
binary_score='yes'
Processing document 6 of 7
binary_score='yes'
Processing document 7 of 7
binary_score='yes'


In [10]:
for entry in history:
    print(f"Q: {entry['question']}")
    print(f"A: {entry['answer']}")

Q: Who is the CEO of Megaforce?(profile)
A: Based on the provided context, it appears that WL_XU is listed as the Chairman and CEO of Megaforce. Therefore, I can inform you that WL_XU is the CEO of Megaforce.
Q: When was Megaforce founded?(profile)
A: I'd be happy to help you with that!

Based on the provided context, I found information relevant to your optimized question.

The company founding date of Megaforce is 1991. This information can be found in the context as "1991\n英濟香港有限公司成立。" which translates to "1991, 英濟香港有限公司 was founded."
Q: 英濟公司的策略長是誰?(profile)
A: 根據提供的資訊，英濟公司的策略長是趙晟。
Q: 英濟公司的執行長是誰?(profile)
A: 根據提供的組織架構資訊，英濟公司的執行長（董事長暨執行長）是徐文麟先生。
Q: 英濟公司什麼時候成立的?(profile)
A: 根據提供的內容，英濟公司於1991年成立。
Q: 英濟公司的事業有什麼?(product)
A: 根據提供的內容，我們可以知道英濟公司事業包括：

* 精密塑膠零組件射出成型
* 微精密結構模具開發
* 電子組裝
* 噴塗印刷等前中後段製程整合服務
* 醫療級矽膠成型技術
* 牙科機電整合產品
* 微創手術器械產品
* 醫療IoT整合產品
* 藥械合一產品
* 醫療教學課程
* 光電事業，包括雷射應用技術、擴增實境(AR)顯示、高端醫學掃描以及數位牙科口腔建模技術

這些都是英濟公司事業的範圍。
Q: 介紹一下英濟公司的高精密塑膠暨矽膠成型事業?(product)
A: 根據提供的背景資訊，英济公司的高精密塑膠暨矽膠成型事業